imports

In [1]:
import pandas as pd
import numpy as np

create dataframe "data" and grab the first lines abstract

In [2]:
# create dataframe
data = pd.read_csv('award_data.csv') # loads and parses award data
data.iloc[0]['Abstract'] # prints out first lines abstract

/tmp/ipykernel_316447/1612965084.py:2: DtypeWarning: Columns (10,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('award_data.csv') # loads and parses award data


"The Department of Homeland Security (DHS) grapples with vast and diverse datasets collected daily, ranging from personal property scans to Stream of Commerce (SoC) data. To analyze and improve algorithms for detecting explosives and prohibited items, efficient curation and labeling are essential. However, DHS faces challenges, including data processing inefficiencies, dependency on human labeling, limited scalability, predictive analytics and threat detection obstacles, and inter-agency collaboration barriers.In response, Agile Data Decisions, Inc. (AgileDD) proposes an innovative solution called AI for Data Labeling and Curation at Scale (AI-DLCS). Leveraging their iQC human-in-the-loop AI platform and the CargoSeer AI platform, the project aims to address DHS's challenges. CargoSeer AI, developed by CargoSeer LTD, specializes in consignment inspection, utilizing a Large Foundation Model to automatically inspect scanned cargo for fraud. AgileDD plans to enhance these platforms with n

normalize data, get rid of data objects with NaN as the state

In [ ]:
# convert state column to uppercase and removes \n and " "
data['State'] = data['State'].str.strip().str.upper()
# get unique states after cleaning
unique_states = data['State'].unique()
print("Unique States After Cleaning:", unique_states)
# replace NAN with pd.NA
data['State'].replace('NAN', pd.NA, inplace=True) 
# removes all rows where state is pd.NA
data.dropna(subset=['State'], inplace=True)
# re-check unique states to verify cleaning
unique_states_cleaned = data['State'].unique()
print("Final Unique States:", unique_states_cleaned)

Final Unique States: ['TX' 'CA' 'VT' 'MD' 'GA' 'MI' 'MA' 'NC' 'FL' 'CO' 'OH' 'CT' 'IL' 'AK'
 'NM' 'VA' 'OR' 'AL' 'NY' 'PA' 'WY' 'WI' 'MN' 'NV' 'IN' 'AZ' 'ID' 'KY'
 'DE' 'NJ' 'SC' 'MT' 'DC' 'MO' 'NE' 'HI' 'ND' 'WA' 'TN' 'UT' 'WV' 'OK'
 'RI' 'AR' 'KS' 'LA' 'IA' 'ME' 'NH' 'MS' 'PR' 'SD' 'AS' 'MH' 'VI']


/tmp/ipykernel_316447/3455209180.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['State'].replace('NAN', pd.NA, inplace=True)


create a dictionary d[state] = list of abstract associated with that state:
Ex: 
{
    "CALIFORNIA": ["Abstract 1", "Abstract 2", "Abstract 3"],
    "TEXAS": ["Abstract A", "Abstract B"],
    "OREGON": ["Abstract X"]
}


In [4]:
state_column = 'State'
proposal_title_column = 'Abstract'
data[state_column] = data[state_column].str.upper()
"""
data.groupby(state_column)
    creates grouped object
    rows are grouped together by their states
data.groupby(state_column)[proposal_title_column]
    select only the title column
data.groupby(state_column)[proposal_title_column].apply(list)
    converts each state's groups of absrtacts into a list
data.groupby(state_column)[proposal_title_column].apply(list).to_dict()
    converts to a dictionary: dict[state] = list of abstract of that state
"""
state_proposals = data.groupby(state_column)[proposal_title_column].apply(list).to_dict()
print(len(state_proposals))

55
